In [46]:
## Get my bucket name
my_bucket  <- Sys.getenv("WORKSPACE_BUCKET")
## Google project name
GOOGLE_PROJECT <- Sys.getenv("GOOGLE_PROJECT")

In [1]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "delirium & dementia" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_84799776_person_sql <- paste("
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `person` person 
    LEFT JOIN
        `concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
person_84799776_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "person_84799776",
  "person_84799776_*.csv")
message(str_glue('The data will be written to {person_84799776_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_84799776_person_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  person_84799776_path,
  destination_format = "CSV")



── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
The data will be written to gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/bq_exports/vraptis@researchallofus.org/20241007/person_84799776/person_84799776_*.csv. Use this path when reading the data into your notebooks in the future.



In [2]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {person_84799776_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(gender = col_character(), race = col_character(), ethnicity = col_character(), sex_at_birth = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_84799776_person_df <- read_bq_export_from_workspace_bucket(person_84799776_path)

dim(dataset_84799776_person_df)

head(dataset_84799776_person_df, 5)

Loading gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/bq_exports/vraptis@researchallofus.org/20241007/person_84799776/person_84799776_000000000000.csv.



[1] 413457     10

person_id,gender_concept_id,gender,date_of_birth,race_concept_id,race,ethnicity_concept_id,ethnicity,sex_at_birth_concept_id,sex_at_birth
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
1036100,1585843,Gender Identity: Additional Options,1996-06-15 00:00:00 UTC,1177221,I prefer not to answer,903079,PMI: Prefer Not To Answer,4124462,None
1938775,1585843,Gender Identity: Additional Options,1986-06-15 00:00:00 UTC,1177221,I prefer not to answer,903079,PMI: Prefer Not To Answer,4124462,None
3028890,1585843,Gender Identity: Additional Options,1982-06-15 00:00:00 UTC,1177221,I prefer not to answer,903079,PMI: Prefer Not To Answer,4124462,None
1944011,1177221,I prefer not to answer,1967-06-15 00:00:00 UTC,1177221,I prefer not to answer,903079,PMI: Prefer Not To Answer,4124462,None
2109271,1177221,I prefer not to answer,1975-06-15 00:00:00 UTC,1177221,I prefer not to answer,903079,PMI: Prefer Not To Answer,4124462,None


In [3]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "delirium & dementia" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_84799776_condition_sql <- paste("
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_end_datetime,
        c_occurrence.condition_type_concept_id,
        c_type.concept_name as condition_type_concept_name,
        c_occurrence.stop_reason,
        c_occurrence.visit_occurrence_id,
        visit.concept_name as visit_occurrence_concept_name,
        c_occurrence.condition_source_value,
        c_occurrence.condition_source_concept_id,
        c_source_concept.concept_name as source_concept_name,
        c_source_concept.concept_code as source_concept_code,
        c_source_concept.vocabulary_id as source_vocabulary,
        c_occurrence.condition_status_source_value,
        c_occurrence.condition_status_concept_id,
        c_status.concept_name as condition_status_concept_name 
    FROM
        ( SELECT
            * 
        FROM
            `condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (4182210)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1) 
                OR  condition_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (35207116, 44833398)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )) c_occurrence 
    LEFT JOIN
        `concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
    LEFT JOIN
        `concept` c_type 
            ON c_occurrence.condition_type_concept_id = c_type.concept_id 
    LEFT JOIN
        `visit_occurrence` v 
            ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
    LEFT JOIN
        `concept` visit 
            ON v.visit_concept_id = visit.concept_id 
    LEFT JOIN
        `concept` c_source_concept 
            ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
    LEFT JOIN
        `concept` c_status 
            ON c_occurrence.condition_status_concept_id = c_status.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
condition_84799776_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "condition_84799776",
  "condition_84799776_*.csv")
message(str_glue('The data will be written to {condition_84799776_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_84799776_condition_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  condition_84799776_path,
  destination_format = "CSV")



The data will be written to gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/bq_exports/vraptis@researchallofus.org/20241007/condition_84799776/condition_84799776_*.csv. Use this path when reading the data into your notebooks in the future.



In [4]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {condition_84799776_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), condition_type_concept_name = col_character(), stop_reason = col_character(), visit_occurrence_concept_name = col_character(), condition_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), condition_status_source_value = col_character(), condition_status_concept_name = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_84799776_condition_df <- read_bq_export_from_workspace_bucket(condition_84799776_path)

dim(dataset_84799776_condition_df)

head(dataset_84799776_condition_df, 5)

Loading gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/bq_exports/vraptis@researchallofus.org/20241007/condition_84799776/condition_84799776_000000000000.csv.



[1] 47188    20

person_id,condition_concept_id,standard_concept_name,standard_concept_code,standard_vocabulary,condition_start_datetime,condition_end_datetime,condition_type_concept_id,condition_type_concept_name,stop_reason,visit_occurrence_id,visit_occurrence_concept_name,condition_source_value,condition_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,condition_status_source_value,condition_status_concept_id,condition_status_concept_name
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1387888,375791,Uncomplicated senile dementia,191449005,SNOMED,2014-08-12 00:00:00 UTC,NA,44786629,Secondary Condition,NA,9.0e+15,Office Visit,290.0,44835772,"Senile dementia, uncomplicated",290.0,ICD9CM,NA,NA,NA
1399242,4182210,Dementia,52448006,SNOMED,1983-03-21 06:00:00 UTC,NA,32821,EHR billing record,NA,4.2e+16,Outpatient Visit,290,44824105,Dementias,290,ICD9CM,NA,NA,NA
2183968,43530666,Dementia with behavioral disturbance,1591000119103,SNOMED,2021-11-08 00:00:00 UTC,2021-11-08 11:59:59 UTC,44786629,Secondary Condition,NA,2.7e+16,Inpatient Visit,F03.91,45595843,Unspecified dementia with behavioral disturbance,F03.91,ICD10CM,NA,NA,NA
1859387,374888,Dementia associated with another disease,191519005,SNOMED,1999-04-30 00:00:00 UTC,1999-04-30 11:59:59 UTC,44786627,Primary Condition,NA,2.7e+16,Outpatient Visit,294.1,44821814,Dementia in conditions classified elsewhere,294.1,ICD9CM,NA,NA,NA
1268114,374888,Dementia associated with another disease,191519005,SNOMED,2019-06-03 14:56:00 UTC,NA,0,No matching concept,NA,4.4e+16,Outpatient Visit,294.11,44827644,Dementia in conditions classified elsewhere with behavioral disturbance,294.11,ICD9CM,NA,NA,NA


In [5]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "delirium & dementia" for domain "drug" and was generated for All of Us Controlled Tier Dataset v7
dataset_84799776_drug_sql <- paste("
    SELECT
        d_exposure.person_id,
        d_exposure.drug_concept_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_standard_concept.concept_code as standard_concept_code,
        d_standard_concept.vocabulary_id as standard_vocabulary,
        d_exposure.drug_exposure_start_datetime,
        d_exposure.drug_exposure_end_datetime,
        d_exposure.verbatim_end_date,
        d_exposure.drug_type_concept_id,
        d_type.concept_name as drug_type_concept_name,
        d_exposure.stop_reason,
        d_exposure.refills,
        d_exposure.quantity,
        d_exposure.days_supply,
        d_exposure.sig,
        d_exposure.route_concept_id,
        d_route.concept_name as route_concept_name,
        d_exposure.lot_number,
        d_exposure.visit_occurrence_id,
        d_visit.concept_name as visit_occurrence_concept_name,
        d_exposure.drug_source_value,
        d_exposure.drug_source_concept_id,
        d_source_concept.concept_name as source_concept_name,
        d_source_concept.concept_code as source_concept_code,
        d_source_concept.vocabulary_id as source_vocabulary,
        d_exposure.route_source_value,
        d_exposure.dose_unit_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN (SELECT
                    DISTINCT ca.descendant_id 
                FROM
                    `cb_criteria_ancestor` ca 
                JOIN
                    (SELECT
                        DISTINCT c.concept_id       
                    FROM
                        `cb_criteria` c       
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id             
                        FROM
                            `cb_criteria` cr             
                        WHERE
                            concept_id IN (21604792)             
                            AND full_text LIKE '%_rank1]%'       ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) b 
                        ON (ca.ancestor_id = b.concept_id)))) d_exposure 
        LEFT JOIN
            `concept` d_standard_concept 
                ON d_exposure.drug_concept_id = d_standard_concept.concept_id 
        LEFT JOIN
            `concept` d_type 
                ON d_exposure.drug_type_concept_id = d_type.concept_id 
        LEFT JOIN
            `concept` d_route 
                ON d_exposure.route_concept_id = d_route.concept_id 
        LEFT JOIN
            `visit_occurrence` v 
                ON d_exposure.visit_occurrence_id = v.visit_occurrence_id 
        LEFT JOIN
            `concept` d_visit 
                ON v.visit_concept_id = d_visit.concept_id 
        LEFT JOIN
            `concept` d_source_concept 
                ON d_exposure.drug_source_concept_id = d_source_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
drug_84799776_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "drug_84799776",
  "drug_84799776_*.csv")
message(str_glue('The data will be written to {drug_84799776_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_84799776_drug_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  drug_84799776_path,
  destination_format = "CSV")



The data will be written to gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/bq_exports/vraptis@researchallofus.org/20241007/drug_84799776/drug_84799776_*.csv. Use this path when reading the data into your notebooks in the future.



In [6]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {drug_84799776_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character(), drug_type_concept_name = col_character(), stop_reason = col_character(), sig = col_character(), route_concept_name = col_character(), lot_number = col_character(), visit_occurrence_concept_name = col_character(), drug_source_value = col_character(), source_concept_name = col_character(), source_concept_code = col_character(), source_vocabulary = col_character(), route_source_value = col_character(), dose_unit_source_value = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
dataset_84799776_drug_df <- read_bq_export_from_workspace_bucket(drug_84799776_path)

dim(dataset_84799776_drug_df)

head(dataset_84799776_drug_df, 5)

Loading gs://fc-secure-0e4de6e0-e2d7-4267-949d-7b1ad758a53f/bq_exports/vraptis@researchallofus.org/20241007/drug_84799776/drug_84799776_000000000000.csv.



[1] 33210    27

person_id,drug_concept_id,standard_concept_name,standard_concept_code,standard_vocabulary,drug_exposure_start_datetime,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,drug_type_concept_name,⋯,lot_number,visit_occurrence_id,visit_occurrence_concept_name,drug_source_value,drug_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,route_source_value,dose_unit_source_value
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>,<chr>,⋯,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1946077,40223769,donepezil hydrochloride 10 MG Oral Tablet [Aricept],997224,RxNorm,2015-07-22 18:23:12.2 UTC,2015-12-21 23:02:38.6 UTC,NA,32839,EHR prescription issue record,⋯,NA,NA,NA,NA,NA,NA,NA,NA,26643006,NA
2487356,733562,rivastigmine 1.5 MG Oral Capsule,314214,RxNorm,2012-01-30 17:13:13.5 UTC,2012-04-23 16:30:09.7 UTC,NA,32839,EHR prescription issue record,⋯,NA,NA,NA,NA,NA,NA,NA,NA,26643006,NA
1373225,40223778,donepezil hydrochloride 5 MG Oral Tablet,997229,RxNorm,2017-10-05 22:00:00 UTC,2017-10-14 20:56:00 UTC,NA,38000177,Prescription written,⋯,NA,2.7e+16,Inpatient Visit,NA,NA,NA,NA,NA,26643006,NA
3423014,40222862,memantine hydrochloride 5 MG Oral Tablet,996571,RxNorm,2020-05-20 09:00:00 UTC,2020-05-22 18:30:00 UTC,NA,38000177,Prescription written,⋯,NA,2.7e+16,Inpatient Visit,NA,NA,NA,NA,NA,26643006,NA
1767904,40223778,donepezil hydrochloride 5 MG Oral Tablet,997229,RxNorm,2017-12-05 22:45:00 UTC,2017-12-06 20:58:00 UTC,NA,38000177,Prescription written,⋯,NA,2.7e+16,Inpatient Visit,NA,NA,NA,NA,NA,26643006,NA


**Process demographics:**

In [7]:
### filter and reformat basic demographics
temp_person_df <-
dataset_84799776_person_df %>% 
    ## select relevant columns
    select(person_id, date_of_birth, sex_at_birth) %>%
    ## reformat dob as date
    mutate(date_of_birth = strptime(date_of_birth, format="%Y-%m-%d")) %>% 
    ## reformat sex_at_birth: 0 female; 1 male; NA other  
    mutate(sex_at_birth = case_when(sex_at_birth=="Female" ~ 0,
                           sex_at_birth=="Male" ~ 1,
                           TRUE ~ NA)) %>%
    ## rename columns
    rename(dob = date_of_birth,
           sex = sex_at_birth)
temp_person_df %>% head

cat("# of people by sex (0=female; 1=male; NA: other):")
temp_person_df$sex %>% as.factor %>% summary

person_id,dob,sex
<dbl>,<dttm>,<dbl>
1036100,1996-06-15,NA
1938775,1986-06-15,NA
3028890,1982-06-15,NA
1944011,1967-06-15,NA
2109271,1975-06-15,NA
1211026,1952-06-15,NA


# of people by sex (0=female; 1=male; NA: other):

0      1   NA's 
249565 155169   8723

**Process conditions:**

In [8]:
dataset_84799776_condition_df %>% head

person_id,condition_concept_id,standard_concept_name,standard_concept_code,standard_vocabulary,condition_start_datetime,condition_end_datetime,condition_type_concept_id,condition_type_concept_name,stop_reason,visit_occurrence_id,visit_occurrence_concept_name,condition_source_value,condition_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,condition_status_source_value,condition_status_concept_id,condition_status_concept_name
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1387888,375791,Uncomplicated senile dementia,191449005,SNOMED,2014-08-12 00:00:00 UTC,NA,44786629,Secondary Condition,NA,9.0e+15,Office Visit,290.0,44835772,"Senile dementia, uncomplicated",290.0,ICD9CM,NA,NA,NA
1399242,4182210,Dementia,52448006,SNOMED,1983-03-21 06:00:00 UTC,NA,32821,EHR billing record,NA,4.2e+16,Outpatient Visit,290,44824105,Dementias,290,ICD9CM,NA,NA,NA
2183968,43530666,Dementia with behavioral disturbance,1591000119103,SNOMED,2021-11-08 00:00:00 UTC,2021-11-08 11:59:59 UTC,44786629,Secondary Condition,NA,2.7e+16,Inpatient Visit,F03.91,45595843,Unspecified dementia with behavioral disturbance,F03.91,ICD10CM,NA,NA,NA
1859387,374888,Dementia associated with another disease,191519005,SNOMED,1999-04-30 00:00:00 UTC,1999-04-30 11:59:59 UTC,44786627,Primary Condition,NA,2.7e+16,Outpatient Visit,294.1,44821814,Dementia in conditions classified elsewhere,294.1,ICD9CM,NA,NA,NA
1268114,374888,Dementia associated with another disease,191519005,SNOMED,2019-06-03 14:56:00 UTC,NA,0,No matching concept,NA,4.4e+16,Outpatient Visit,294.11,44827644,Dementia in conditions classified elsewhere with behavioral disturbance,294.11,ICD9CM,NA,NA,NA
2615147,377254,"Multi-infarct dementia, uncomplicated",70936005,SNOMED,2014-06-26 00:00:00 UTC,NA,44786629,Secondary Condition,NA,9.0e+15,Office Visit,290.40,44827641,"Vascular dementia, uncomplicated",290.40,ICD9CM,NA,NA,NA


In [9]:
dataset_84799776_condition_df %>% 
    mutate(source_concept_name_code = paste0(source_concept_name, "_", source_concept_code)) %>%      
    group_by(source_concept_name_code) %>% summarise(n=n()) %>% arrange(desc(n)) %>%
    head(.,10)

source_concept_name_code,n
<chr>,<int>
Unspecified dementia without behavioral disturbance_F03.90,8078
Postconcussional syndrome_F07.81,6757
Dementia in other diseases classified elsewhere without behavioral disturbance_F02.80,5445
Postconcussion syndrome_310.2,3651
"Alzheimer's disease, unspecified_G30.9",3119
Delirium due to known physiological condition_F05,2957
Dementia in other diseases classified elsewhere with behavioral disturbance_F02.81,2263
Vascular dementia without behavioral disturbance_F01.50,2116
Alzheimer's disease with late onset_G30.1,1454


In [10]:
## get earliest delirium date
delirium_concept_codes <- c("F05", "293.0")

temp_delirium_df <-
dataset_84799776_condition_df %>%
    ## extract cases for delirium not induced by alcohol/other substances
    filter(source_concept_code %in% delirium_concept_codes) %>%
    ## reformat starting date variable
    mutate(delirium_date = strptime(condition_start_datetime, format="%Y-%m-%d")) %>% 
    ## select relevant columns: id, delirium_code, delirium_date
    select(person_id, source_concept_code, delirium_date, source_concept_name_code) %>%
    ## extract only the first delirium instance per person
    group_by(person_id) %>%
    mutate(delirium_count = n()) %>% # calculate number of delirium instances per person
    arrange(delirium_date) %>% 
    slice(1L) %>% 
    ## make delirium status columns
    mutate(delirium_status = 1) %>%
    ## rename columns
    rename(delirium_code = source_concept_code)
   
cat("# of delirium instances:", sum(temp_delirium_df$delirium_count), "\n")
cat("# of unique delirium indivuals:", nrow(temp_delirium_df))

temp_delirium_df %>% head


# of delirium instances: 3939 
# of unique delirium indivuals: 1889

person_id,delirium_code,delirium_date,delirium_count,delirium_status
<dbl>,<chr>,<dttm>,<int>,<dbl>
1000320,F05,2017-07-27,1,1
1001121,F05,2020-05-15,3,1
1002832,F05,2019-11-19,1,1
1002867,F05,2017-03-19,1,1
1003498,F05,2016-02-10,1,1
1004234,F05,2020-11-19,1,1


In [91]:
## get individuals with AD + dates 
alzheimer_full <- 
dataset_84799776_condition_df %>%
    # AD status + date
    mutate(alzheimer_status = ifelse(str_detect(source_concept_name, "Alzheimer"),1,0),
           alzheimer_date   = ifelse(str_detect(source_concept_name, "Alzheimer"), condition_start_datetime, NA),
           alzheimer_date   = strptime(alzheimer_date, format="%Y-%m-%d")
          ) %>%
    filter(alzheimer_status==1) %>% 
    # extract only the first AD record per person
    group_by(person_id) %>%
    #mutate(dementia_count = n()) %>% 
    arrange(alzheimer_date) %>% 
    slice(1L) %>%
    select(person_id, source_concept_name, source_concept_code, starts_with("alz"))
alzheimer_full %>% dim
alzheimer_full %>% head

[1] 888   5

person_id,source_concept_name,source_concept_code,alzheimer_status,alzheimer_date
<dbl>,<chr>,<chr>,<dbl>,<dttm>
1001959,Alzheimer's disease with late onset,G30.1,1,2020-07-30
1004198,Alzheimer's disease,331.0,1,2012-12-10
1006354,"Alzheimer's disease, unspecified",G30.9,1,2021-01-19
1008635,"Alzheimer's disease, unspecified",G30.9,1,2016-07-27
1009326,"Alzheimer's disease, unspecified",G30.9,1,2022-02-24
1010116,"Alzheimer's disease, unspecified",G30.9,1,2018-03-21


In [28]:
## get earliest dementia time from diagnoses
dementia_concept_codes <- setdiff(unique(dataset_84799776_condition_df$source_concept_code), delirium_concept_codes) 

temp_dementia_df <- 
dataset_84799776_condition_df %>%
    ## extract cases for dementia
    filter(source_concept_code %in% dementia_concept_codes) %>%
    ## reformat starting date variable
    mutate(dementia_date = strptime(condition_start_datetime, format="%Y-%m-%d")) %>%     
    ## select relevant columns: id, dementia_code, dementia_date
    select(person_id, source_concept_code, dementia_date, source_concept_name) %>%
    ## extract only the first dementia record per person
    group_by(person_id) %>%
    mutate(dementia_count = n()) %>% 
    arrange(dementia_date) %>% 
    slice(1L) %>%
    ## make dementia status column
    mutate(dementia_status = 1) %>% 
    ## rename columns
    rename(dementia_code = source_concept_code) 
   
cat("# of unique dementia indivuals:", nrow(temp_dementia_df))

temp_dementia_df %>% head


# of unique dementia indivuals: 5679

person_id,dementia_code,dementia_date,source_concept_name,dementia_count,dementia_status
<dbl>,<chr>,<dttm>,<chr>,<int>,<dbl>
1000079,F07.81,2015-12-18,Postconcussional syndrome,1,1
1000291,290.13,2010-12-15,Presenile dementia with depressive features,7,1
1000801,F03.90,2020-11-20,Unspecified dementia without behavioral disturbance,1,1
1000926,F07.81,2020-06-03,Postconcussional syndrome,2,1
1000976,F07.81,2017-06-13,Postconcussional syndrome,4,1
1001395,310.2,1999-10-22,Postconcussion syndrome,1,1


**Process drugs table:**

In [29]:
dataset_84799776_drug_df %>% group_by(standard_concept_name) %>% summarise(n=n()) %>% arrange(desc(n)) %>% head(.,10)

standard_concept_name,n
<chr>,<int>
donepezil hydrochloride 10 MG Oral Tablet,6644
donepezil hydrochloride 5 MG Oral Tablet,5260
memantine hydrochloride 10 MG Oral Tablet,4485
donepezil,3719
memantine hydrochloride 5 MG Oral Tablet,2366
memantine,2361
donepezil hydrochloride 10 MG Oral Tablet [Aricept],1154
donepezil hydrochloride 5 MG Oral Tablet [Aricept],881
memantine hydrochloride 10 MG Oral Tablet [Namenda],880


In [30]:
dataset_84799776_drug_df %>% head(.,2)

person_id,drug_concept_id,standard_concept_name,standard_concept_code,standard_vocabulary,drug_exposure_start_datetime,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,drug_type_concept_name,⋯,lot_number,visit_occurrence_id,visit_occurrence_concept_name,drug_source_value,drug_source_concept_id,source_concept_name,source_concept_code,source_vocabulary,route_source_value,dose_unit_source_value
<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<dbl>,<chr>,⋯,<chr>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
1946077,40223769,donepezil hydrochloride 10 MG Oral Tablet [Aricept],997224,RxNorm,2015-07-22 18:23:12.2 UTC,2015-12-21 23:02:38.6 UTC,NA,32839,EHR prescription issue record,⋯,NA,NA,NA,NA,NA,NA,NA,NA,26643006,NA
2487356,733562,rivastigmine 1.5 MG Oral Capsule,314214,RxNorm,2012-01-30 17:13:13.5 UTC,2012-04-23 16:30:09.7 UTC,NA,32839,EHR prescription issue record,⋯,NA,NA,NA,NA,NA,NA,NA,NA,26643006,NA


In [31]:
## get anti dementia drug start date

temp_drugs_df <- 
dataset_84799776_drug_df %>%
    ## reformat starting date variable
    mutate(drug_date = strptime(drug_exposure_start_datetime, format="%Y-%m-%d")) %>% 
    ## select relevant columns: id, drug code, drug date
    select(person_id, standard_concept_code, drug_date) %>%
    ## extract only the first drug exposure date
    group_by(person_id) %>%
    mutate(drug_count = n()) %>% # calculate number of drug istances
    arrange(drug_date) %>% 
    slice(1L) %>% 
    ## make drug status column
    mutate(drug_status = 1) %>%
    ## rename
    rename(drug_code = standard_concept_code)
 

**Merge:**

In [32]:
## merge dementia and anti-dementia drug tables
temp_dementia_df2 <- 
full_join(temp_dementia_df, temp_drugs_df, by="person_id") %>%
    mutate(earliest_dementia_date = case_when( drug_date < dementia_date  ~ drug_date,
                                               drug_date > dementia_date ~ dementia_date,
                                               drug_date == dementia_date ~ dementia_date,
                                               is.na(drug_date) ~ dementia_date, # when only diagnosis is available
                                               is.na(dementia_date) ~ drug_date, # when only drug is available
                                               TRUE ~ NA )) %>% 
    mutate(earliest_dementia_source = case_when( drug_date < dementia_date  ~ paste0("drug", "_", drug_code),
                                                 drug_date > dementia_date ~ paste0("condition", "_", dementia_code),
                                                 drug_date == dementia_date ~ paste0("both", "_", dementia_code, "_", drug_code),
                                                 is.na(drug_date) ~ paste0("condition", "_", dementia_code), # when only diagnosis is available
                                                 is.na(dementia_date) ~ paste0("drug", "_", drug_code), # when only drug is available
                                                 TRUE ~ NA ))

dim(temp_dementia_df2)
head(temp_dementia_df2)

[1] 7047   12

person_id,dementia_code,dementia_date,source_concept_name,dementia_count,dementia_status,drug_code,drug_date,drug_count,drug_status,earliest_dementia_date,earliest_dementia_source
<dbl>,<chr>,<dttm>,<chr>,<int>,<dbl>,<chr>,<dttm>,<int>,<dbl>,<dttm>,<chr>
1000079,F07.81,2015-12-18,Postconcussional syndrome,1,1,NA,NA,NA,NA,2015-12-18,condition_F07.81
1000291,290.13,2010-12-15,Presenile dementia with depressive features,7,1,NA,NA,NA,NA,2010-12-15,condition_290.13
1000801,F03.90,2020-11-20,Unspecified dementia without behavioral disturbance,1,1,997223,2020-12-03,17,1,2020-11-20,condition_F03.90
1000926,F07.81,2020-06-03,Postconcussional syndrome,2,1,NA,NA,NA,NA,2020-06-03,condition_F07.81
1000976,F07.81,2017-06-13,Postconcussional syndrome,4,1,NA,NA,NA,NA,2017-06-13,condition_F07.81
1001395,310.2,1999-10-22,Postconcussion syndrome,1,1,NA,NA,NA,NA,1999-10-22,condition_310.2


In [34]:
## merge delirium & dementia tables and make dementia incident variable 
# -1 = dementia before delirium (prevalent); 
# 0 = same date as delirium; 
# 1 = after delirium (incident)

deldem <- 
full_join(temp_delirium_df, temp_dementia_df2, by="person_id") %>%
    mutate(dementia_incident = case_when( delirium_date < earliest_dementia_date  ~ 1,
                                          delirium_date > earliest_dementia_date ~ -1,
                                          delirium_date == earliest_dementia_date ~ 0,
                                          TRUE ~ NA))


cat("# of individuals wiht dementia OR delirium:", nrow(deldem))
head(deldem)

# of individuals wiht dementia OR delirium: 8502

person_id,delirium_code,delirium_date,delirium_count,delirium_status,dementia_code,dementia_date,source_concept_name,dementia_count,dementia_status,drug_code,drug_date,drug_count,drug_status,earliest_dementia_date,earliest_dementia_source,dementia_incident
<dbl>,<chr>,<dttm>,<int>,<dbl>,<chr>,<dttm>,<chr>,<int>,<dbl>,<chr>,<dttm>,<int>,<dbl>,<dttm>,<chr>,<dbl>
1000320,F05,2017-07-27,1,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1001121,F05,2020-05-15,3,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1002832,F05,2019-11-19,1,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1002867,F05,2017-03-19,1,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1003498,F05,2016-02-10,1,1,F03.90,2017-04-28,Unspecified dementia without behavioral disturbance,3,1,997229,2016-07-22,68,1,2016-07-22,drug_997229,1
1004234,F05,2020-11-19,1,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [35]:
deldem %>% group_by(delirium_status, dementia_incident) %>% summarise(n=n())

`summarise()` has grouped output by 'delirium_status'. You can override using
the `.groups` argument.


delirium_status,dementia_incident,n
<dbl>,<dbl>,<int>
1,-1,225
1,0,71
1,1,138
1,NA,1455
NA,NA,6613


In [36]:
deldem %>% 
    group_by(delirium_status) %>% 
    summarise(n=n(),
              dementia_prev = round((sum(dementia_status==1,na.rm = F)/n())*100,2))

delirium_status,n,dementia_prev
<dbl>,<int>,<dbl>
1,1889,NA
NA,6613,NA


In [37]:
## merge with all participants
full <- left_join(temp_person_df, deldem, by = "person_id") %>%
    mutate(delirium_status = ifelse(is.na(delirium_status),0,delirium_status)) %>%
    mutate(dementia_status = ifelse(is.na(dementia_status),0,dementia_status)) %>%
    mutate(dementia_incident = ifelse(is.na(dementia_incident),-9,dementia_incident))


In [38]:
## get participants with genomic data from ancestry table
## Copy ancestry_preds.tsv table to workspace
library(data.table)
GOOGLE_PROJECT <- Sys.getenv("GOOGLE_PROJECT")
system(paste("gsutil -u", GOOGLE_PROJECT, "cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv ."), intern=T)
## load ancestry table
ancestry_pred <- fread("ancestry_preds.tsv") %>% select(research_id, ancestry_pred_other) %>% rename(person_id = research_id)


character(0)

In [39]:
full %>% 
    group_by(delirium_status) %>% 
    summarise(n_del =n(), 
              `dementia (%)` = round((sum(dementia_status==1)/n())*100,2),
              `dementia_prev (%)` = round((sum(dementia_incident==0 | dementia_incident==-1)/n())*100,2),
              `dementia_inc (%)` = round((sum(dementia_incident==1)/n())*100,2)              
              )
 

delirium_status,n_del,dementia (%),dementia_prev (%),dementia_inc (%)
<dbl>,<int>,<dbl>,<dbl>,<dbl>
0,411568,1.29,0.00,0.00
1,1889,20.65,15.67,7.31


In [52]:
full_gen <-
full %>% 
    filter(person_id %in% ancestry_pred$person_id) %>%
    rename("dementia_concept_name" = source_concept_name)
full_gen %>%
    group_by(delirium_status) %>% 
    summarise(n_del =n(), 
              `dementia (%)` = round((sum(dementia_status==1)/n())*100,2),
              `dementia_prev (%)` = round((sum(dementia_incident==-1 | dementia_incident==0)/n())*100,2),
              `dementia_inc (%)` = round((sum(dementia_incident==1)/n())*100,2)              
              )
 

delirium_status,n_del,dementia (%),dementia_prev (%),dementia_inc (%)
<dbl>,<int>,<dbl>,<dbl>,<dbl>
0,244250,1.46,0.00,0.00
1,1138,20.56,16.43,6.33


In [92]:
write.table(full_gen, "full_with_dementia.txt", sep=" ", row.names=F, col.names=T, quote=T)
write.table(alzheimer_full, "full_with_AD.txt", sep=" ", row.names=F, col.names=T, quote=T)

In [93]:
system(paste0("gsutil cp ./", "full_with_dementia.txt", " ", my_bucket, "/data/pheno/with_dementia/full.txt"), intern=T)
system(paste0("gsutil cp ./", "full_with_AD.txt", " ", my_bucket, "/data/pheno/with_dementia/full_AD.txt"), intern=T)


character(0)

character(0)